In [3]:
import pandas as pd
import numpy as np

ffs = [
    '成品电导率', 
    '成品PH值', 
    '成品浓度', 
    '成品浊度',
    '成品温度', 
    '成品表面张力', 
    '成品COD', 
    '单耗',
    '柠檬酸添加比例', 
    '置换量', 
    '离子液添加量', 
    '原液添加比例', 
    '异常类型',
]

df = pd.read_csv('../data/切割液数据0914.csv', sep='\t', usecols=ffs+['钢线类型'], low_memory=False)
df = df[df['钢线类型'] == 'JU-C-32'].copy()
df.pop('钢线类型')

print(f'初始数据量：{len(df)}')

features = [ 
            '成品电导率', 
            '成品PH值', 
            '成品浓度', 
            '成品浊度',
            '成品温度', 
            '成品表面张力', 
            '成品COD', 
            '单耗'
            ]

error_type = ['断缝', '跳线', '']
df = df[df['异常类型'].isin(error_type) | df['异常类型'].isna()].copy()
print(f'断缝/正常/跳线数据量：{len(df)}')
df['异常类型2'] = df['异常类型'].apply(lambda x: x if x in ['断缝', '跳线'] else '正常')
df['异常类型'] = df['异常类型'].apply(lambda x: 1 if x in ['断缝', '跳线'] else 0)
df['柠檬酸添加比例'] = df['柠檬酸添加比例'].replace('#VALUE!', np.nan)


df.replace('#DIV/0!', np.nan, inplace=True)
df.dropna(how='any', inplace=True)
for c in df.columns:
    if c not in ['异常类型2', '钢线类型'] :
        try:
            df[c] = df[c].astype(float)
        except Exception as e:
            print(c, e)
            df[c] = df[c].apply(lambda x: float(x.replace('%', ''))/100)

初始数据量：11552
断缝/正常/跳线数据量：10597
成品浓度 could not convert string to float: '1.28%'
置换量 could not convert string to float: '6.000%'
离子液添加量 could not convert string to float: '0.00%'


## 分箱

需要刀数，统计异常率，拟合异常率


In [4]:
import pandas as pd
from optbinning import OptimalBinning
import numpy as np
import pwlf # type: ignore
from sklearn.linear_model import Ridge
import copy
import itertools

# global bs
# bs = dict()

def apply_optimal_binning(data, feature, target):
    '''
    使用 OptimalBinning 对数据进行分箱处理
    :data: 数据
    :feature: 特征
    :target: 目标变量
    '''
    
    opt_bin = OptimalBinning(name=feature, dtype="numerical", min_bin_size=0.005, min_bin_n_event=10)
    opt_bin.fit(data[feature], target)
    
    # 获取并显示分箱区间
    binning_splits = opt_bin.splits
    
    # 获取最小值和最大值
    min_val = data[feature].min()
    max_val = data[feature].max()
    
    # 构建完整的区间
    intervals = [min_val] + list(binning_splits) + [max_val]
    interval_pairs = [(intervals[i], intervals[i+1]) for i in range(len(intervals)-1)]
    
    def get_interval(x):
        for _, (a, b) in enumerate(interval_pairs):
            if a <= x <= b:
                return str([round(a, 4), round(b, 4)])
        return str([])
    
    binned_data_interval = data[feature].apply(get_interval)
    
    def get_data(x):
        for _, (a, b) in enumerate(interval_pairs):
            if a <= x <= b:
                return (a+b)/2
        return -1

    binned_data_mean = data[feature].apply(get_data)
    
    return binned_data_mean, binned_data_interval


def remark(dataall, features=features, features_formul=['柠檬酸添加比例', '置换量', '离子液添加量', '原液添加比例']):
    '''
    对数据进行分箱处理，并使用线性回归模型进行拟合
    :data: 数据
    :features: 成品特征
    :features_formul: 配方特征
    '''
    data = dataall[features + ['异常类型']].copy()
    # X 是特征，y 是目标变量
    X1 = data.drop(columns=['异常类型'])
    X2 = copy.deepcopy(X1)
    y = data['异常类型']

    # 对每个特征进行分箱并替换原始数据
    for col in X1.columns:
        X1[col], X2[col] = apply_optimal_binning(data, col, y)
    
    # 选择特征
    features2 = [x + '区间' for x in features]
    X2 = X2[features]
    X2.columns = features2
    
    featuresplus = features + features2
    
    # 拟合单个元素的作用
    # 返回modelpwlfs
    modelpwlfs = dict()
    dataxy = pd.concat([X1, X2, y], axis=1)
    for col in features:
        dataxyg = dataxy[[col, '异常类型']].groupby(col).mean().reset_index() # 单个特征与异常类型的均值
        dataxyg.sort_values(col, inplace=True)
        r = pwlf.PiecewiseLinFit(dataxyg[col], dataxyg['异常类型'])
        r.fit(len(dataxyg))
        modelpwlfs[col] = r
        
    # 拟合单个元素产生的y值
    dataxy_cov = dataxy.groupby(featuresplus).agg(
        异常率统计=('异常类型', 'mean'), 
        刀数=('异常类型', 'count')
        ).reset_index() # 多个特征与异常类型的均值
    dataxy_cov_pwlf = copy.deepcopy(dataxy_cov)

    # 转化为用区间均值预测的数据
    for col in features:
        dataxy_cov_pwlf[col] = modelpwlfs[col].predict(dataxy_cov_pwlf[col].values)

    r = Ridge(fit_intercept=False)
    r.fit(dataxy_cov_pwlf[features], dataxy_cov_pwlf['异常率统计'])
    
    # 寻找最好的成品值
    dataxy_cov['异常率估计'] = r.predict(dataxy_cov_pwlf[features])
    dataxy_cov['刀数'] = dataxy_cov['刀数']
    dataxy_cov.sort_values('异常率估计', ascending=True, inplace=True)

    # 寻找最最佳配方
    def get_best_formula(x, features_formul):
        '''
        获取最佳配方
        :data: 数据
        :features_formul: 配方特征
        '''
        best_quality = x.to_dict()
        data_formul = pd.concat([X2, dataall[features_formul]], axis=1)
        for k, v in best_quality.items():
            data_formul = data_formul[data_formul[k] == v]
        
        data_formul.sort_values(by=features_formul[:-1], inplace=True, ascending=True)
        return data_formul[features_formul].iloc[0].T.to_dict()
    
    dataxy_cov['配比'] = dataxy_cov.apply(lambda x: get_best_formula(x[features2], features_formul), axis=1)
    return dataxy_cov[features2 + ['异常率统计', '异常率估计', '刀数', '配比']]



In [5]:
data1 = df[df['异常类型2'].isin(['断缝'])].copy()
data11 = df[df['异常类型2'].isin(['正常'])].copy()
data1 = pd.concat([data1, data11], axis=0)
data1 = data1[ffs]
data1 = data1.sample(frac=1).reset_index(drop=True)
print('断缝最优比例')
data = remark(data1)
data.to_csv('../model/断缝最优比例.csv', index=False)
data.head(5)

断缝最优比例


,成品电导率区间,成品PH值区间,成品浓度区间,成品浊度区间,成品温度区间,成品表面张力区间,成品COD区间,单耗区间,异常率统计,异常率估计,刀数,配比
687,"[13.365, 14.77]","[5.36, 5.545]","[0.0126, 0.0129]","[57.95, 63.695]","[25.15, 25.55]","[27.85, 28.05]","[23241.25, 23843.75]","[2.815, 3.055]",0.000000,0.075055,11,"{'柠檬酸添加比例': 1.45, '置换量': 0.06, '离子液添加量': 0.0, ..."
558,"[9.075, 13.365]","[5.545, 6.095]","[0.0126, 0.0129]","[13.11, 51.05]","[25.55, 26.35]","[28.05, 28.155]","[22972.5, 23241.25]","[2.815, 3.055]",0.000000,0.075760,8,"{'柠檬酸添加比例': 0.0, '置换量': 0.04, '离子液添加量': 0.06, ..."
511,"[8.895, 9.075]","[6.155, 6.29]","[0.013, 0.0132]","[63.695, 97.92]","[25.55, 26.35]","[28.05, 28.155]","[22972.5, 23241.25]","[2.815, 3.055]",0.000000,0.075862,24,"{'柠檬酸添加比例': 0.0, '置换量': 0.04, '离子液添加量': 0.06, ..."
547,"[9.075, 13.365]","[5.545, 6.095]","[0.0126, 0.0129]","[13.11, 51.05]","[25.55, 26.35]","[27.85, 28.05]","[22972.5, 23241.25]","[2.815, 3.055]",0.076923,0.075923,26,"{'柠檬酸添加比例': 0.0, '置换量': 0.04, '离子液添加量': 0.06, ..."
699,"[13.365, 14.77]","[5.36, 5.545]","[0.0126, 0.0129]","[63.695, 97.92]","[25.55, 26.35]","[27.85, 28.05]","[23241.25, 23843.75]","[2.815, 3.055]",0.000000,0.076012,12,"{'柠檬酸添加比例': 1.45, '置换量': 0.06, '离子液添加量': 0.0, ..."


In [6]:
data1 = df[df['异常类型2'].isin(['跳线'])].copy()
data11 = df[df['异常类型2'].isin(['正常'])].copy()
data1 = pd.concat([data1, data11], axis=0)
data1 = data1[ffs]
data1 = data1.sample(frac=1).reset_index(drop=True)
print('跳线最优比例')
data = remark(data1)
data.to_csv('../model/跳线最优比例.csv', index=False)
data.head(5)

跳线最优比例


,成品电导率区间,成品PH值区间,成品浓度区间,成品浊度区间,成品温度区间,成品表面张力区间,成品COD区间,单耗区间,异常率统计,异常率估计,刀数,配比
629,"[9.155, 9.475]","[6.115, 6.135]","[0.013, 0.0132]","[32.945, 35.6]","[25.75, 26.15]","[28.05, 28.155]","[23707.5, 23858.75]","[0.09, 2.635]",0.125000,0.109815,8,"{'柠檬酸添加比例': 0.0, '置换量': 0.0451, '离子液添加量': 0.06..."
51,"[6.55, 7.415]","[6.115, 6.135]","[0.0129, 0.013]","[32.945, 35.6]","[26.15, 26.35]","[28.05, 28.155]","[24077.5, 25875.0]","[2.635, 4.025]",0.000000,0.110920,6,"{'柠檬酸添加比例': 0.0, '置换量': 0.0451, '离子液添加量': 0.06..."
50,"[6.55, 7.415]","[6.115, 6.135]","[0.0129, 0.013]","[32.945, 35.6]","[26.15, 26.35]","[27.95, 28.05]","[24077.5, 25875.0]","[2.635, 4.025]",0.066667,0.110956,15,"{'柠檬酸添加比例': 0.0, '置换量': 0.0451, '离子液添加量': 0.06..."
14,"[6.55, 7.415]","[6.045, 6.115]","[0.0126, 0.0129]","[32.945, 35.6]","[24.95, 25.75]","[27.95, 28.05]","[24077.5, 25875.0]","[4.565, 40.0]",0.142857,0.112238,7,"{'柠檬酸添加比例': 0.0, '置换量': 0.05, '离子液添加量': 0.06, ..."
56,"[6.55, 7.415]","[6.115, 6.135]","[0.0129, 0.013]","[35.6, 47.105]","[24.95, 25.75]","[28.05, 28.155]","[24077.5, 25875.0]","[4.565, 40.0]",0.000000,0.112555,13,"{'柠檬酸添加比例': 0.0, '置换量': 0.05, '离子液添加量': 0.06, ..."


In [7]:
data1 = df[df['异常类型2'].isin(['跳线', '断缝'])].copy()
data11 = df[df['异常类型2'].isin(['正常'])].copy()
data1 = pd.concat([data1, data11], axis=0)
data1 = data1[ffs]
data1 = data1.sample(frac=1).reset_index(drop=True)
print('跳线+断缝最优比例')
data = remark(data1)
data.to_csv('../model/跳线+断缝最优比例.csv', index=False)
data.head(5)

跳线+断缝最优比例


,成品电导率区间,成品PH值区间,成品浓度区间,成品浊度区间,成品温度区间,成品表面张力区间,成品COD区间,单耗区间,异常率统计,异常率估计,刀数,配比
408,"[8.985, 9.155]","[5.36, 5.925]","[0.0125, 0.0127]","[32.05, 35.6]","[26.35, 27.9]","[27.85, 28.05]","[23103.75, 24235.0]","[4.565, 40.0]",0.000000,0.177637,5,"{'柠檬酸添加比例': 1.82, '置换量': 0.05, '离子液添加量': 0.0, ..."
411,"[8.985, 9.155]","[5.36, 5.925]","[0.0127, 0.0129]","[32.05, 35.6]","[26.35, 27.9]","[27.85, 28.05]","[23103.75, 24235.0]","[4.565, 40.0]",0.500000,0.177943,2,"{'柠檬酸添加比例': 0.91, '置换量': 0.05, '离子液添加量': 0.0, ..."
419,"[8.985, 9.155]","[5.925, 5.985]","[0.0127, 0.0129]","[35.6, 46.6]","[26.35, 27.9]","[27.85, 28.05]","[23103.75, 24235.0]","[4.565, 40.0]",0.285714,0.178936,7,"{'柠檬酸添加比例': 0.91, '置换量': 0.05, '离子液添加量': 0.0, ..."
418,"[8.985, 9.155]","[5.925, 5.985]","[0.0127, 0.0129]","[35.6, 46.6]","[25.15, 26.15]","[27.85, 28.05]","[22972.5, 23103.75]","[0.675, 4.565]",0.285714,0.179705,7,"{'柠檬酸添加比例': 0.0, '置换量': 0.04, '离子液添加量': 0.06, ..."
481,"[9.155, 9.475]","[5.985, 6.135]","[0.0125, 0.0127]","[32.05, 35.6]","[26.35, 27.9]","[27.85, 28.05]","[23103.75, 24235.0]","[4.565, 40.0]",0.307692,0.179748,13,"{'柠檬酸添加比例': 0.91, '置换量': 0.05, '离子液添加量': 0.0, ..."


## 聚类

In [8]:
# mdf = pd.DataFrame(data['配比'].tolist())

# # 聚类
# from sklearn.cluster import KMeans
# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import silhouette_score

# scaler = StandardScaler()
# X = mdf.values
# X = scaler.fit_transform(X)
# n_clusters = range(2, 15)
# silhouette = []
# for n in n_clusters:
#     kmeans = KMeans(n_clusters=n, random_state=0)
#     kmeans.fit(X)
#     silhouette.append(silhouette_score(X, kmeans.labels_))

# import plotly.express as px
# fig = px.line(x=n_clusters, y=silhouette, title='最佳聚类数')
# fig.show()


In [9]:
data['异常率统计'] = data['异常率统计'] * data['刀数']
data['异常率估计'] = data['异常率估计'] * data['刀数']
data['配比'] = data['配比'].apply(lambda x: str(x))
datag = data.groupby('配比').agg(
    刀数=('刀数', 'sum'),
    异常率统计=('异常率统计', 'sum'),
    异常率估计=('异常率估计', 'sum')
    ).reset_index()

datag['异常率统计'] = datag['异常率统计'] / datag['刀数']
datag['异常率估计'] = datag['异常率估计'] / datag['刀数']
datag

,配比,刀数,异常率统计,异常率估计
0,"{'柠檬酸添加比例': 0.0, '置换量': 0.04, '离子液添加量': 0.06, ...",1096,0.142336,0.187834
1,"{'柠檬酸添加比例': 0.0, '置换量': 0.0451, '离子液添加量': 0.06...",201,0.134328,0.189041
2,"{'柠檬酸添加比例': 0.0, '置换量': 0.0451, '离子液添加量': 0.06...",732,0.210383,0.201383
3,"{'柠檬酸添加比例': 0.0, '置换量': 0.05, '离子液添加量': 0.0, '...",170,0.182353,0.193914
4,"{'柠檬酸添加比例': 0.0, '置换量': 0.05, '离子液添加量': 0.06, ...",264,0.196970,0.190916
5,"{'柠檬酸添加比例': 0.0, '置换量': 0.05, '离子液添加量': 0.06, ...",1146,0.183246,0.197094
6,"{'柠檬酸添加比例': 0.0, '置换量': 0.05, '离子液添加量': 0.06, ...",1946,0.220966,0.203362
7,"{'柠檬酸添加比例': 0.0, '置换量': 0.05, '离子液添加量': 0.07, ...",222,0.252252,0.205682
8,"{'柠檬酸添加比例': 0.0, '置换量': 0.05, '离子液添加量': 0.07, ...",1488,0.241263,0.198097
9,"{'柠檬酸添加比例': 0.0, '置换量': 0.0547, '离子液添加量': 0.06...",57,0.192982,0.188326
